In [ ]:
#Not needed in Zeppelin
from pyspark.sql import *

spark = SparkSession.builder.appName("dataCleaning").getOrCreate()

In [ ]:
import sys
from csv import reader
from pyspark.sql.functions import *

In [ ]:
# Final csv should be styled as such
# ('VendorID', 'pickup_datetime', 'dropoff_datetime', 'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'congestion_surcharge')

# Move data from green taxi set around to match above comment
def mapGreenTaxi(x):
    return (x[0], x[1], x[2], x[7], x[8], x[4], x[3], x[5], x[6], x[17], x[9], x[10], x[11], x[12], x[13], x[15], x[16], x[19])

# Ensure data from yellow taxi set is consistent with the new green taxi data format
def mapYellowTaxi(x):
    return (x[0], x[1], x[2], x[3], x[4], x[5], x[6], x[7], x[8], x[9], x[10], x[11], x[12], x[13], x[14], x[15], x[16], x[17])

# Remove the header lines
def filterHeaders(x):\
    if (x[0] != "VendorID"):
        return x
        
# Remove any data with a negative trip_distance or fare_amount
def removeInvalidValues(x):
    if float(x[4]) > 0 and float(x[10]) > 0:
        return x
    
# Join list by comma to create CSV lines
def formatAsCSV(x):
    return ','.join(x)

In [ ]:
green_taxi = sc.textFile("2020_Green_Taxi_Trip_Data.csv", 1)
yellow_taxi = sc.textFile("2020_Yellow_Taxi_Trip_Data.csv", 1)
green_taxi = green_taxi.mapPartitions(lambda x : reader(x)).map(mapGreenTaxi).filter(filterHeaders)
yellow_taxi = green_taxi.mapPartitions(lambda x : reader(x)).map(mapYellowTaxi).filter(filterHeaders)

taxi = green_taxi.union(yellow_taxi).filter(removeInvalidValues).map(formatAsCSV)

In [ ]:
taxi.saveAsTextFile("2020_Taxi_Trip_Data.out")

In [ ]:
#Not needed in Zeppelin
sc.close()